In [1]:
!pip install openpyxl 
!pip install geopandas
!pip install shapely


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
^C
Traceback (most recent call last):
  File "/root/venv/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/root/venv/lib/python3.9/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  F

In [2]:
import warnings
# Not recommenden for production.
warnings.filterwarnings('ignore')

Start on library importation

In [22]:
#To work with dataframes and geopands to handle geojsons
import pandas as pd
import geopandas as gpd
import numpy as np
import json 

#Shapely and scipy to work with vectors and geostadistics
from shapely.geometry import shape, Polygon, Point
from scipy.spatial.distance import euclidean


In [4]:

#We are going to use this module later, for good practices must be at the begging
# Calculate the center of the polygon, later determine which is closest finding the CUSEC and towns respectivelly

def calcular_centroide(row):
    geometry_data = row['geometry']
    #Convertion of the data on a recognizable shape for Shapely.
    multipolygon = shape(geometry_data)
    #Take the centroid and the coordinate on X and Y
    centroid = multipolygon.centroid
    centroid_coords = (centroid.x, centroid.y)  # Convert our centroide on a tuple to use later for calculations
    return centroid_coords


In [5]:
poblacion_sec_censal = pd.read_csv('/work/GPX programing test/poblacion_sec_censal.csv', sep= ';')
#population of all census sections of the province of Barcelona identified by their unique census section identifier.
print(poblacion_sec_censal.info(), '\n \n')
print(poblacion_sec_censal.head(10))


# Here we must change the name Secci�n  which has troubles with the encoding route and drop columns without value
poblacion_sec_censal.rename(columns={'Secci�n': 'Seccion', }, inplace=True)
poblacion_sec_censal = poblacion_sec_censal.drop(columns=['Sexo','Edad (grupos quinquenales)'])

print('\n DF modified \n',poblacion_sec_censal.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3613 entries, 0 to 3612
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Sexo                        3613 non-null   object 
 1   Secci�n                     3613 non-null   int64  
 2   Edad (grupos quinquenales)  3613 non-null   object 
 3   Total                       3613 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 113.0+ KB
None 
 

          Sexo    Secci�n Edad (grupos quinquenales)    Total
0  Ambos Sexos  800101001                      Total    1.497
1  Ambos Sexos  800101002                      Total    1.868
2  Ambos Sexos  800101003                      Total  835.000
3  Ambos Sexos  800101004                      Total    1.922
4  Ambos Sexos  800101005                      Total    1.844
5  Ambos Sexos  800101007                      Total    2.627
6  Ambos Sexos  800101008                    

In [7]:
#Here we have to face the design of the sheet and only extract the headers that start at row 7 
atlas_socioeconimo = pd.read_excel('/work/GPX programing test/atlas_socioeconomico.xlsx', header=7)


print(atlas_socioeconimo.info(), '\n \n')
print(atlas_socioeconimo.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55095 entries, 0 to 55094
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0           55092 non-null  object
 1   2017    55087 non-null  object
 2   2016    55087 non-null  object
 3   2015    55087 non-null  object
 4   2017.1  55087 non-null  object
 5   2016.1  55087 non-null  object
 6   2015.1  55087 non-null  object
 7   2017.2  55087 non-null  object
 8   2016.2  55087 non-null  object
 9   2015.2  55087 non-null  object
 10  2017.3  55087 non-null  object
 11  2016.3  55087 non-null  object
 12  2015.3  55087 non-null  object
 13  2017.4  55087 non-null  object
 14  2016.4  55087 non-null  object
 15  2015.4  55087 non-null  object
dtypes: object(16)
memory usage: 6.7+ MB
None 
 

                                               2017   2016   2015 2017.1  \
0                     01001 Alegría-Dulantzi  13281  13086  12936  34618   
1       0100101 Alegría-Dulantzi distrito

In [9]:
#Extract only the columns that we are interested, on this case only Renta neta media por persona 2017,2016, 2015
atlas_socioeconimo = atlas_socioeconimo.iloc[:, [0, 1, 2, 3]]

#Change columnas name to get clear infomation

atlas_socioeconimo.columns = ['Index', '2017 Income', '2016 Income', '2015 Income']

#Change datatypes to filter with dropna ah use only the rows that contain information
atlas_socioeconimo['2017 Income'] = pd.to_numeric(atlas_socioeconimo['2017 Income'], errors='coerce').astype('Int64')
atlas_socioeconimo['2016 Income'] = pd.to_numeric(atlas_socioeconimo['2016 Income'], errors='coerce').astype('Int64')
atlas_socioeconimo['2015 Income'] = pd.to_numeric(atlas_socioeconimo['2015 Income'], errors='coerce').astype('Int64')
atlas_socioeconimo = atlas_socioeconimo.fillna(0)
#Create the global variable taking in consideration all income in this period
atlas_socioeconimo['Global'] =atlas_socioeconimo['2017 Income']+atlas_socioeconimo['2016 Income']+atlas_socioeconimo['2015 Income']


#Reduction of the df removing values without data
atlas_socioeconimo = atlas_socioeconimo[atlas_socioeconimo['Global']!=0]


# Take codes appart from location name because we need to do Joins in the future
#atlas_socioeconimo['NPRO'] = atlas_socioeconimo['Index'].str.split().str[-1]
atlas_socioeconimo['Index'] = atlas_socioeconimo['Index'].str.extract('(\d+)')


#Obtain incomes and codes
atlas_socioeconimo

,Index,2017 Income,2016 Income,2015 Income,Global
0,01001,13281,13086,12936,39303
4,01002,13862,13691,13800,41353
13,01003,16177,17194,15786,49157
16,01004,12530,13185,12722,38437
19,01006,13115,11330,13040,37485
...,...,...,...,...,...
55082,5200108011,16433,16373,15847,48653
55083,5200108012,17350,17185,16792,51327
55084,5200108013,12553,12705,11823,37081
55085,5200108014,8906,8872,8937,26715


In [10]:
#Create our df using geopandas
zonas_operador = gpd.read_file('/work/GPX programing test/zonas_operador.geojson')
print(zonas_operador.info(), '\n \n')
print(zonas_operador.head(10))

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Index       113 non-null    int64   
 1   Coloration  113 non-null    int64   
 2   C0          113 non-null    float64 
 3   C1          113 non-null    float64 
 4   ZE          113 non-null    object  
 5   Surface     113 non-null    float64 
 6   TauxConcav  113 non-null    float64 
 7   Lac-CellId  113 non-null    object  
 8   Population  113 non-null    object  
 9   geometry    113 non-null    geometry
dtypes: float64(4), geometry(1), int64(2), object(3)
memory usage: 9.0+ KB
None 
 

   Index  Coloration        C0        C1      ZE  Surface  TauxConcav  \
0  10113         246  0.614314  0.154390  10113;     4.10    0.147437   
1  10115         248  0.721709  0.260285  10115;     0.80    0.122245   
2  10114         247  0.572262  0.343758  10114;     0.51    0.122252   
3  1012

In [11]:
# Apply our function to get the centroides and create a new column with them
zonas_operador['centroide'] = zonas_operador.apply(calcular_centroide, axis=1)

#Taking only in consideration usefull columns
zonas_operador = zonas_operador[['Index', 'Population','centroide']]
zonas_operador.Index = zonas_operador.Index.astype('object')


#Check if is alredy done
zonas_operador

,Index,Population,centroide
0,10113,17520,"(2.1569322495068772, 41.36446822555657)"
1,10115,38090,"(2.159359923207574, 41.37853652453478)"
2,10114,22000,"(2.1688545577838925, 41.373011178875345)"
3,10125,22923,"(2.172905132941295, 41.376540848891736)"
4,10124,14739,"(2.190197608860213, 41.377349237279056)"
...,...,...,...
108,1102,91083,"(2.03636781029606, 41.3430572426467)"
109,1159,66168,"(2.032205213860239, 41.302976277777645)"
110,949,46705,"(1.9837474451185597, 41.294668219954396)"
111,880,6961,"(1.9015356228294058, 41.31860011293308)"


In [12]:
#As always, create our df
zonas_censales_ine = gpd.read_file('/work/GPX programing test/zonas_censales_ine.json')

print(zonas_censales_ine.info(), '\n \n')
print(zonas_censales_ine.head(10))

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36309 entries, 0 to 36308
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   CUSEC       36309 non-null  object  
 1   CUMUN       36309 non-null  object  
 2   CSEC        36309 non-null  object  
 3   CDIS        36309 non-null  object  
 4   CMUN        36309 non-null  object  
 5   CPRO        36309 non-null  object  
 6   CCA         36309 non-null  object  
 7   CUDIS       36309 non-null  object  
 8   CLAU2       36309 non-null  object  
 9   NPRO        36309 non-null  object  
 10  NCA         36309 non-null  object  
 11  CNUT0       36309 non-null  object  
 12  CNUT1       36309 non-null  object  
 13  CNUT2       36309 non-null  object  
 14  CNUT3       36309 non-null  object  
 15  ESTADO      36309 non-null  object  
 16  OBS         36309 non-null  object  
 17  NMUN        36309 non-null  object  
 18  Shape_Leng  36309 non-null  float64 
 

In [13]:
#Create an interactive df, take in consideration here I am using deepnote which allows me this feature

zonas_censales_ine

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,...,CNUT0,CNUT1,CNUT2,CNUT3,ESTADO,OBS,NMUN,Shape_Leng,Shape_Area,geometry
0,0100901001,01009,001,01,009,01,16,0100901,01009,Araba/Álava,...,ES,2,1,1,I,,Asparrena,51725.353538,6.326080e+07,"MULTIPOLYGON (((-2.25069 42.89550, -2.31610 42..."
1,0101001002,01010,002,01,010,01,16,0101001,01010,Araba/Álava,...,ES,2,1,1,I,,Ayala/Aiara,13350.774728,7.332951e+06,"POLYGON ((-2.97499 43.09904, -3.01602 43.11833..."
2,0103101001,01031,001,01,031,01,16,0103101,01031,Araba/Álava,...,ES,2,1,1,I,,Laguardia,87711.717051,8.041601e+07,"MULTIPOLYGON (((-2.52467 42.61471, -2.55285 42..."
3,0103301001,01033,001,01,033,01,16,0103301,01033,Araba/Álava,...,ES,2,1,1,I,,Lapuebla de Labarca,12331.494377,5.950453e+06,"POLYGON ((-2.54903 42.48445, -2.57448 42.50752..."
4,0103701001,01037,001,01,037,01,16,0103701,01037,Araba/Álava,...,ES,2,1,1,I,,Arraia-Maeztu,60761.315212,1.227207e+08,"POLYGON ((-2.36929 42.80425, -2.40900 42.80644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,3501605017,35016,017,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",15642.540059,5.318911e+06,"POLYGON ((-15.49347 28.08960, -15.49699 28.096..."
36305,3501605009,35016,009,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",2929.633264,2.023513e+05,"POLYGON ((-15.48841 28.09245, -15.48927 28.093..."
36306,3501605033,35016,033,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",9796.053160,2.625522e+06,"POLYGON ((-15.48999 28.08495, -15.49145 28.086..."
36307,3501605034,35016,034,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",1994.804489,1.635899e+05,"POLYGON ((-15.48999 28.08495, -15.48559 28.089..."


In [24]:
#Start our variable with columns for my new df, a empty list to add more list and create a df, and an empty list to add errors
columnas = ['Index', 'Population', 'CUSEC', 'NPRO', 'NCA', 'NMUN']
ndf = []
faltantes= []

#Start to run our indexes looking for closest point
for elemento in zonas_operador['Index']:

    #To avoid use to much compute power fisrt we filter zonas_censales_ine reducing the number of operations
    resultado = zonas_censales_ine[zonas_censales_ine['CUSEC'].str.contains(str(elemento))]
    #create a variable for our centroid
    centroide_str = zonas_operador[zonas_operador['Index'] == elemento]['centroide']
    #Same for population because we will need in the future to complement our df
    population = zonas_operador[zonas_operador['Index'] == elemento]['Population']
    # Obtain the numeric value
    population = population.iloc[0]



    if resultado.shape[0] == 0:
        faltantes.append(elemento)


    else:
        centroide = centroide_str.iloc[0]  # Obtain the numeric value
        #Apply the function to our df results
        resultado['centroide'] = resultado.apply(calcular_centroide, axis=1)

        #Calculation of euclidean distance btw our centroid and each column
        resultado['distancia'] = resultado['centroide'].apply(lambda x: euclidean(centroide, x))
        
        # Find the minimum index 
        indice_fila_mas_cercana = resultado['distancia'].idxmin()
        
        # Append the closest values to our list of list
        fila_mas_cercana = resultado.loc[indice_fila_mas_cercana]
        #Element because this was the ennuncement
        #Population to calculate the global
        # CUSEC for look in atlas_socioeconimo
        #NPRO to check cities looking potential joins
        #NCA to check cities looking potential joins
        #NMUN to check cities looking potential joins

        ndf.append([str(elemento),population,fila_mas_cercana['CUSEC'],fila_mas_cercana['NPRO'],fila_mas_cercana['NCA'],fila_mas_cercana['NMUN'] ])


In [80]:
#Creation of df features
columnas = ['Index', 'Population', '2017 Income','2016 Income','2015 Income']
df = pd.DataFrame(columns=columnas)



for lista in ndf:

    cusec_mc = lista[2]
    #Check if CUSEC is in our DF with best granularity
    if cusec_mc in atlas_socioeconimo['Index'].values:
        
        fila = {'Index': lista[0]  ,
                  '2017 Income':  atlas_socioeconimo[atlas_socioeconimo['Index'] == cusec_mc]['2017 Income'].iloc[0] ,
                  '2016 Income':   atlas_socioeconimo[atlas_socioeconimo['Index'] == cusec_mc]['2016 Income'].iloc[0],
                  '2015 Income':  atlas_socioeconimo[atlas_socioeconimo['Index'] == cusec_mc]['2015 Income'].iloc[0]} 
        df = df.append(fila, ignore_index=True)        
    #If dont exist, we could go further and check on the CUMUN to get the data 
    else:
        first_five_digits = cusec_mc[:5]
        
        if first_five_digits in atlas_socioeconimo['Index'].str[:5].values:
                fila = {'Index': lista[0]  ,
                        '2017 Income':  atlas_socioeconimo[atlas_socioeconimo['Index'] == first_five_digits]['2017 Income'].iloc[0] ,
                        '2016 Income':   atlas_socioeconimo[atlas_socioeconimo['Index'] == first_five_digits]['2016 Income'].iloc[0],
                        '2015 Income':  atlas_socioeconimo[atlas_socioeconimo['Index'] == first_five_digits]['2015 Income'].iloc[0]} 
                df = df.append(fila, ignore_index=True)   
        #If we dont have any data at this point we consider that these data is not available
        else:
            
                fila = {'Index': lista[0]  ,
                         '2017 Income': np.nan ,
                          '2016 Income':   np.nan ,
                          '2015 Income':  np.nan} 
                df = df.append(fila, ignore_index=True)
  


In [82]:
#The average is taken for each year, since the file with the request does not clarify whether it is the average of the years with data or the average for each year
df.to_csv('archivo_solicitdado.csv', index=False) 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d79b2b15-9a76-4b8a-9139-6103e6385a17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>